In [1]:
%%capture
!"C:\Program Files\Python310\python.exe" -m pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 torch
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" "C:\Program Files\Python310\python.exe" -m pip install llama-cpp-python

## LLM und Tokenizer laden

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    #"./models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85",
    device_map="auto", 
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")#"./models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85",)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Übung
## Aufgabe 1:
Erzeugen Sie einen one_shot_prompt bei dem Sie das Wort "Gedankenexperiment" als user und assistent erklären. Anschließend machen Sie ein Beispiel zum  Wort "Brezn" und lassen Sie sich darauf hin ein Satz mit Brezn bilden.

In [ ]:
one_shot_prompt = [

]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

In [ ]:
# Erzeugung des Outputs
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

 During the intense duel, the knight skillfully screeged his opponent's shield, forcing him to defend himself.


## Aufgabe 2: Chain Prompting
Erzeugen Sie mit Hilfe mehrere Prompts einen Namen und einen Slogan für einen Brezn Verkaufsautomaten.
Nutzen Sie anschließend das Ergebnis um hierfür eine Produktbeschreibung in drei Sätzen zu generieren.


In [ ]:
product_prompt = [

]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

In [ ]:
sales_prompt = [

]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

 Introducing MindMeld Messenger - your ultimate communication partner! Unleash intelligent conversations with our innovative AI-powered messaging platform. With MindMeld Messenger, every response is thoughtful, personalized, and timely. Say goodbye to generic replies and hello to meaningful interactions. Elevate your communication game with MindMeld Messenger - where every message is a step towards smarter conversations. Try it now and experience the future of messaging!


## Aufgabe3: Zero-shot Chain-of-Thought
Folgende Angabe ist gegeben:

Ein Metzger verkauft Weißwürste. Er hat 3 Pfund Weißwürste, die pro Pfund 4,50 € kosten und 240g Butter pro Pfund enthalten. Pro Weißwurst sind 12g Schweinefleisch verarbeitet. 1 Pfund (ca. 454g) ist ein 10-teiliges Paket. Eine Weißwurst wiegt ca. 50g. Ein Kunde möchte 10 Weißwürste kaufen, aber er hat nur 15 € dabei und möchte noch ein 250g Päckchen Butter dazu kaufen, das 1,80 € kostet. 

Nutzen Sie das LLM um eine oder mehrere der folgenden Fragen zu lösen. Funktioniert die Lösung mit oder ohne aufzeichnung der Schritte besser?

1. Wie viele Weißwürste sind in den 3 Pfund, die der Metzger hat?
2. Wie viel Gramm Fleisch sind in einer Weißwurst enthalten?
3. Wie viel wiegen die 10 Weißwürste, die der Kunde kaufen möchte, und wie viel kostet das?
4. Kauft der Kunde genug Weißwürste für ein Pfund, und ist es genug Fleisch für eine Mahlzeit für ihn und seine Familie?
5. Wie viel Geld bekommt der Metzger, wenn der Kunde die Weißwürste und die Butter bezahlt?
6. Was ist die Gesamtzahl an benötigten Weißwürsten für einen Einkauf von 2 Paketen à 1 Pfund plus Butter für ein 500g Paket?

## Aufgabe 4: Tree-of-Thought: 
Erstellen Sie einen Prompt der folgende Bedingungen erfüllt:
- 3 Experten diskutieren
- Jeder Experte denkt Schrittweise denken.
- Die Experten teilen und vergleichen ihre Ergebnisse bevor sie zum nächsten Schritt wechseln
- Sobald ein Experte falsch liegt meldet, dieser sein falsches Ergebnis und scheidet anschließend aus.
- Die Gruppe soll folgende Frage beantworten:
  Wenn Sie 26 Brezn besitzen und 20 Brezn für das Mittagessen verbrauchen und 6 Brezn neu kaufen,"
    " wie viele Brezn haben Sie dann
- Die Experten sollen Ihr die Ergebnisse diskutieren.


## Aufgabe 5: Output Verification
Generiere ein Brezn Rezept in JSON format.